In [1]:
import subprocess
import shlex
import os

from subprocess import PIPE
from pathlib import Path

In [2]:
def submit_independent_spatially_explicit(repeats=10, walltime=1, speciation=0.000001, sample=0.00025, config='()', output='./STDIN'):
    walltime = "{:02d}".format(1)
    
    config = f"""
    #PBS -lselect={1}:ncpus={1}:mem={2}gb
    #PBS -J 0-{max(repeats - 1, 0)}
    #PBS -lwalltime={walltime}:00:00

    $HOME/necsim-rust/target-base/release/rustcoalescence simulate '(
        speciation: {speciation},
        sample: {sample},
        seed: '$(python3 -c "import random; print(random.getrandbits(64))")',

        algorithm: Independent{config},

        scenario: SpatiallyExplicit(
            habitat: "'$HOME'/necsim-rust/maps/madingley/fg0size12/habitat.tif",
            dispersal: "'$HOME'/necsim-rust/maps/madingley/fg0size12/dispersal.tif",
        ),

        reporters: [
            Plugin(
                library: "'$HOME'/necsim-rust/target-base/release/deps/libnecsim_plugins_common.so",
                reporters: [
                    Execution(),
                    Biodiversity(),
                ],
            ),
        ],
    )'
    """
    
    output = Path(output).resolve(strict=False)
    output.parent.mkdir(parents=True, exist_ok=True)
    
    cwd = os.getcwd()
    os.chdir(output.parent)
    
    # Submit the simulation batch
    result = subprocess.run(shlex.split(
        f"qsub -N {output.name}"
    ), check=True, input=config, stdout=PIPE, universal_newlines=True).stdout.strip()
    
    os.chdir(cwd)
    
    return result

In [3]:
for delta_t in [0.1, 1.0, 2.0]:
    submit_independent_spatially_explicit(
        repeats=10, speciation=0.01, sample=0.00025, config=f"(delta_t:{delta_t})", output=f"delta-t/pbs.{delta_t}"
    )